# Informe Tarea 1 - Beer Clustering

### Integrantes
    - Matías Pacheco
    - Juan Pablo Castillo

## Descripción del dataset

El dataset Beer Reviews es una colección de 1.590.892 críticas de cervezas que se encuentran disponibles en el sitio http://beeradvocate.com/. Éstas críticas toman en consideración características como aroma, apariencia y sabor para asignarle un puntaje a la cerveza que está siendo evaluada.

Cada registro del dataset completo está compuesto por las siguientes columnas:

|Columna             |Significado            |Tipo       |
|---                 |---                    |---        |
|brewery_id          |ID cervecería          |Numérico   |
|brewery_name        |Nombre Cervecería      |Texto      |
|review_time         |Tiempo de evaluación   |Numérico   |
|review_overall      |Puntaje del review     |Numérico   |
|review_aroma        |Puntaje aroma          |Numérico   |
|review_appearence   |Puntaje apariencia     |Numérico   |
|review_profilename  |Nombre del evaluador   |Texto      |
|beer_style          |Tipo de cerveza        |Texto      |
|review_palate       |Puntaje paladar        |Numérico   |
|review_taste        |Puntaje gusto          |Numérico   |
|beer_abv            |Grado alcohólico       |Numérico   |
|beer_beerid         |ID cerveza             |Numérico   |

## Análisis de Clustering

Se realizará un análisis de clustering con el dataset con cada uno de los siguientes cinco algoritmos:
* K-Means
* Minibatch K-Means
* HAC Complete
* Ward
* DBScan

### K-Means

### Minibatch K-Means

### HAC Complete

### Ward

### DBScan


## Conclusiones

¿Qué atributo, Nombre de cervecería o Tipo de cerveza, describe mejor a los cluster como etiquetas
de clase, según los resultados obtenidos previamente? ¿Hay mejores marcas que otras en
relación a las evaluaciones obtenidas? ¿Se puede definir algún criterio para determinar el mejor
tipo de cerveza? Comente